# Finetuning Yolov11l

In [1]:
from ultralytics import YOLO
import os

# load the pretrained large model
model = YOLO("yolo11l.pt")

# Use absolute path to YAML
yaml_path = "/home/joey/Projects/chatbrain/backend/vision_model/chat_dataset.yaml"

# Verify paths
assert os.path.exists(yaml_path), "YAML file not found!"
assert os.path.exists(os.path.join(os.path.dirname(yaml_path), "dataset/train/images")), "Train images missing!"
assert os.path.exists(os.path.join(os.path.dirname(yaml_path), "dataset/val/images")), "Val images missing!"

results = model.train(
    data=yaml_path,
    epochs=200,
    time=8, # maxium training time in hours
    device="0",
    patience=50, # early stopping patience
    batch=3,  # batch size
    multi_scale=False,
    freeze=15,  # Freeze first 15 layers
    mosaic=0.3,  # Reduce augmentation memory usage
    imgsz=640, # image size
    cache=True, # cache images for faster training
)

AssertionError: Train images missing!

#### Sort detected boxes vertically to get message order

In [6]:
def process_results(results):
    boxes = []
    for box in results[0].boxes:
        cls = int(box.cls)
        coords = box.xyxy[0].tolist()  # [ax, ay, bx, by]
        boxes.append((cls, coords))
    
    # Sort non-contact boxes by Y-coordinate (ay)
    contact_boxes = [b for b in boxes if b[0] == 0]
    message_boxes = sorted(
        [b for b in boxes if b[0] != 0],
        key=lambda x: x[1][1]  # Sort by ay (top Y-coordinate)
    )
    
    # Assign 'num' (0 for contacts, 1,2,3... for messages)
    final = []
    for cls, (ax, ay, bx, by) in contact_boxes:
        final.append([ax, ay, bx, by, 0, 0])  # Type 0, num=0
    
    for i, (cls, (ax, ay, bx, by)) in enumerate(message_boxes, 1):
        final.append([ax, ay, bx, by, cls, i])  # Type 1/2, num=i
    
    return final